In [33]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

In [34]:
includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### General Parameters

In [35]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
all_res, plants, parts = read_data(filepath_Ljungan)

(Reservoir[Flasjon
, Holsmjon
], HydropowerPlant[Flasjo
, Trangfors
, Ratan
, Turinge
, Bursnas
, Jarnvagsforsen
, Parteboda
, Hermansboda
, Ljunga
, Nederede
, Skallbole
, Matfors
, Viforsen
], Participant[Sydkraft
, Fortum
, Statkraft
])

### Have a look at the river system we are dealing with    
We will work with Sydkraft in this example

In [36]:
j = parts[1]
res = filter(r -> j.participationrate[r] > 0, all_res)
plantsj = [j.plants[1]]

println("Producer: ", j.name, " \n Reservoirs: ", res[1], "\n Power Plants: ", plantsj[1])

Producer: Sydkraft 
 Reservoirs: Flasjon

 Power Plants: Flasjo



### Set Parameters necessary for Input into Model

In [37]:
Stages = 2
Hours = 1
PricePoints = [0.0, 1.0]
I = length(PricePoints)-1

println("Stages :" , Stages, "\n Hours: ", Hours, "\n I : ", I, "\n Price Points = ", [i for i in PricePoints])
Qref = Dict{Reservoir, Float64}(r => 1.0 for r in res)
scenario_count = 1
Prices = [floor.(rand(Hours), sigdigits=3) for i in 1:scenario_count]
Inflows = [0.0]
Omega = [(price = p, inflow = v) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]
# StartUp Constraints
S = 0.1
mu_up = 10
mu_down = 10

Stages :2
 Hours: 1
 I : 1
 Price Points = [0.0, 1.0]


10

In [56]:
function subproblem_builder(subproblem::Model, node::Int)
    # State Variables
    @variables(subproblem, begin
        0 <= lreal[r = res] <= r.maxvolume, (SDDP.State, initial_value = r.currentvolume)
        # lind[r = res], (SDDP.State, initial_value = r.currentvolume)
        Qnom[r = res] <= Hours * 10, (SDDP.State, initial_value = 0)
        x[i = 1:I+1, t = 1:Hours] <= sum(0.5 * k.spillreference *k.equivalent for  k in plantsj), (SDDP.State, initial_value = 0)
    end)
    # Control Variables
    @variables(subproblem, begin
        zup[t = 1:Hours] >= 0
        zdown[t = 1:Hours] >= 0
        deltastart[k = plantsj, t = 1:Hours], (Bin)
        # deltaind[r = res], (Bin)
        u[k = plantsj, t = 1:Hours], (Bin)
        w[k = plantsj, t = 1:Hours] <= 0.5 * k.spillreference * k.equivalent
        Qnomchange[r = res] >= 0
        0 <= xchange[i = 1:I+1, t = 1:Hours] <= sum(0.5 * k.spillreference * k.equivalent for k in plantsj)
        0 <= Qreal[r = res, t = 1:Hours]
        0 <= y[t = 1:Hours] <= sum(0.5 * k.spillreference * k.equivalent for k in plantsj)
    end)
    # Random Variables
    @variables(subproblem, begin
        Qinflow[r = res]
    end)
    if node == 1
        # Transition Function
        for r in res
            # Real and Individual Reservoir Balance
            @constraint(subproblem, lreal[r].out == lreal[r].in)
            # @constraint(subproblem, lind[r].out == lind[r].in)
            @constraint(subproblem, Qnom[r].out == Qnomchange[r])
        end
        for t in 1:Hours
            for i in 1:I
                @constraint(subproblem, x[i,t].out == xchange[i,t])
                @constraint(subproblem, xchange[i,t] <= xchange[i+1,t])
            end
            @constraint(subproblem, x[I+1,t].out == xchange[I+1,t])
        end
        # Constraints
        for r in res
            @constraint(subproblem, Hours * Qnomchange[r] <= lreal[r].in)
            # Negative individual Reservoir Complications
            # @constraint(subproblem, deltaind[r] => {Qnomchange[r] <= Qref[r]})
            # @constraint(subproblem, !deltaind[r] => {0 <= lind[r].in})
        end
        @stageobjective(subproblem, 0)
    else
        # Transition Function
        for r in res
            # Real and Individual Reservoir Balances
            @constraint(subproblem, lreal[r].out == lreal[r].in - Hours * (Qnom[r].in - Qinflow[r]))
            # @constraint(subproblem, lind[r].out == lind[r].in - Hours * (Qnom[r].in - Qref[r]))
            @constraint(subproblem, Qnom[r].out == Qnomchange[r])
        end
        for t in 1:Hours
            for i in 1:I+1
                @constraint(subproblem, x[i,t].out == xchange[i,t])
            end
        end
        # Constraints
        for t in 1:Hours
            for i in 1:I
                # Increasing bidding Curve constraint
                @constraint(subproblem, xchange[i,t] <= xchange[i+1,t])
            end
            # Fulfill Demand Constraint
            # @constraint(subproblem, y[t] == sum(w[k,t] for k in plantsj) + zup[t] - zdown[t])
            for k in plantsj
                # Power Generation Constraint
                @constraint(subproblem, w[k,t] <= sum(Qreal[r,t] for r in find_us_reservoir(k.reservoir)) * k.equivalent)
                # On-Off and Maximum Generation Constraint
                @constraint(subproblem, u[k,t] * 0 <= w[k,t])
                @constraint(subproblem, w[k,t] <= u[k,t] * 0.5 * k.equivalent * k.spillreference)
                if t > 1
                    # Start-Up
                    @constraint(subproblem, deltastart[k, t] >= u[k,t] - u[k,t-1])
                end
            end
        end

        for r in res
            # Average Discharge Constraint
            @constraint(subproblem, sum(Qreal[r,t] for t in 1:Hours) == Qnom[r].in)
            # Negative Individual Reservoir Complications
            # @constraint(subproblem, deltaind[r] => {Qnomchange[r] <= Qref[r]})
            # @constraint(subproblem, !deltaind[r] => {0 <= lind[r].in})
        end
        # Parametrize Uncertainty
        SDDP.parameterize(subproblem, Omega, P) do om
            for r in res
                JuMP.fix(Qinflow[r], om.inflow)
            end
            for t in 1:Hours
                count = 0
                for i in 1:I
                    if (om.price[t] <= PricePoints[i+1]) && (om.price[t] >= PricePoints[i])
                        # Market Clearing Constraint
                        @constraint(subproblem, y[t] == ((om.price[t] - PricePoints[i])/(PricePoints[i+1] - PricePoints[i]))*x[i+1,t].in + ((PricePoints[i+1] - om.price[t])/(PricePoints[i+1] - PricePoints[i]))*x[i,t].in)
                        count += 1
                    end
                end
                @assert(count == 1)
            end
            # Stage-objective
            @stageobjective(
                subproblem, sum(y[t] * om.price[t] - sum(S * deltastart[k, t] for k in plantsj) - (mu_up * zup[t] + mu_down * zdown[t]) for t in 1:Hours)
                )
        end
    end
    return
end


subproblem_builder (generic function with 1 method)

In [57]:
for i in 1:I
    println(PricePoints[i])
end
for el in Omega
    println(el)
end



Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)
0.0
(price = [0.656], inflow = 0.0)


In [58]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder;
    stages = Stages,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

A policy graph with 2 nodes.
 Node indices: 1, 2


### Train the model

In [59]:
# Lets see if the deterministic equvialent is feasible

det_equiv = SDDP.deterministic_equivalent(model, CPLEX.Optimizer)
optimize!(det_equiv)
JuMP.write_to_file(det_equiv, "det_equiv.lp")

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Dual infeasible due to empty column 'x15'.
Presolve time = 0.00 sec. (0.00 ticks)


In [60]:
println(objective_value(det_equiv))
println(primal_status(det_equiv))
println(dual_status(det_equiv))
println(JuMP.termination_status(det_equiv))

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [61]:
SDDP.train(model; iteration_limit = 100, duality_handler = SDDP.LagrangianDuality())

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 2
  state variables : 4
  scenarios       : 1.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [20, 20]
  VariableRef in MOI.EqualTo{Float64}     : [1, 1]
  VariableRef in MOI.LessThan{Float64}    : [9, 9]
  VariableRef in MOI.ZeroOne              : [2, 2]
  AffExpr in MOI.LessThan{Float64}        : [2, 4]
  VariableRef in MOI.GreaterThan{Float64} : [8, 9]
  AffExpr in MOI.EqualTo{Float64}         : [4, 6]
numerical stability report
  matrix range     [9e-02, 1e+00]
  objective range  [1e-01, 1e+01]
  bounds range     [9e-02, 1e+05]
  rhs range        [0e+00, 0e+00]
----------------------------

         1L  3.868721e-02  5.897440e-02  5.120001e-01         4   1


         2L  5.897440e-02  5.897440e-02  1.150000e+00         8   1

┌ Warning: Found a cut with a mix of small and large coefficients.
│     The order of magnitude difference is 17.09354742726223.
│     The smallest cofficient is -3.469446951953614e-18.
│     The largest coefficient is 0.43033600000000005.
│ 
│ You can ignore this warning, but it may be an indication of numerical issues.
│ 
│ Consider rescaling your model by using different units, e.g, kilometers instead
│ of meters. You should also consider reducing the accuracy of your input data (if
│ you haven't already). For example, it probably doesn't make sense to measure the
│ inflow into a reservoir to 10 decimal places.
└ @ SDDP C:\Users\lenna\.julia\packages\SDDP\gpgyh\src\plugins\bellman_functions.jl:68



         3L  5.897440e-02  5.897440e-02  1.728000e+00        12   1


...

        96L  5.897440e-02  5.897440e-02  6.141500e+01       384   1


        97L  5.897440e-02  5.897440e-02  6.208000e+01       388   1


        98L  5.897440e-02  5.897440e-02  6.273900e+01       392   1


        99L  5.897440e-02  5.897440e-02  6.340900e+01       396   1


       100L  5.897440e-02  5.897440e-02  6.406600e+01       400   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 6.406600e+01
total solves   : 400
best bound     :  5.897440e-02
simulation ci  :  5.877153e-02 ± 3.976290e-04
numeric issues : 0
-------------------------------------------------------------------



### Obtain the Bidding and Nomination Decisions

In [62]:
rule = SDDP.DecisionRule(model; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = merge(Dict(Symbol("lreal[$(r)]") => r.currentvolume for r in res), Dict(Symbol("lind[$(r)]") => r.currentvolume for r in res)),
    controls_to_record = [:Qnom, :y, :zup, :zdown]
)


KeyError: KeyError: key Symbol("lind[Flasjon\n]") not found